In [56]:
import tensorflow as tf
import numpy as np

### 計算 Gradient

在 TensorFlow 中透過 tf.GadientTape 紀錄 Operation 執行的順序，並透過 Reverse Mode Differentiation 計算某一個變數的 Gradient。

In [3]:
x = tf.Variable(4.0)

with tf.GradientTape() as tape:
    y = x**2

當 GradientTape 紀錄完所有的 Operation 後，就可以透過 TradientTape.gradient(target, source) 計算 target 相對於 source 的 gradient。

target => Loss

source => Variable

In [4]:
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())

8.0


前面的 x 為 scalar，亦可是矩陣的運算。

In [5]:
w = tf.Variable(tf.random.normal((3, 2)), name="w")
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name="b")
x = [[1, 2, 3]]

In [6]:
w

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.924929  ,  0.35700756],
       [-0.9914171 ,  0.6693337 ],
       [-0.6632118 , -1.3393688 ]], dtype=float32)>

In [7]:
b

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [8]:
with tf.GradientTape(persistent=True) as tape:
    y = x @ w + b
    loss = tf.reduce_mean(y ** 2)

計算 loss 相對於 w 與 b 的 Gradient

In [9]:
[dl_dw, dl_db] = tape.gradient(loss, [w, b])

gradient 的 shape 會與 source 的 shape 相同！

In [10]:
dl_dw

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ -4.897399 ,  -2.3224316],
       [ -9.794798 ,  -4.644863 ],
       [-14.692197 ,  -6.9672947]], dtype=float32)>

In [11]:
dl_db

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-4.897399 , -2.3224316], dtype=float32)>

也可以將 source 打包為 dictionary 丟入 tape.gradient()

In [12]:
my_vars = {
    'w': w,
    'b': b
}

grad = tape.gradient(loss, my_vars)

In [13]:
grad['w']

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ -4.897399 ,  -2.3224316],
       [ -9.794798 ,  -4.644863 ],
       [-14.692197 ,  -6.9672947]], dtype=float32)>

In [14]:
grad['b']

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-4.897399 , -2.3224316], dtype=float32)>

### 計算 Loss 相對於 Module (Layer or Model) 的 Gradient
通常我們會計算 loss 相對於 module (layer or model) 中的所有 "trainable" variable 的 gradient。

In [21]:
layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1, 2, 3]])

with tf.GradientTape() as tape:
    y = layer(x)
    loss = tf.reduce_mean(y ** 2)

In [22]:
grad = tape.gradient(loss, layer.trainable_variables)

In [23]:
grad

[<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[ 3.8588805,  1.9355916],
        [ 7.717761 ,  3.8711832],
        [11.576641 ,  5.8067746]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([3.8588805, 1.9355916], dtype=float32)>]

### 控制 GradientTape 紀錄的東西

一般來說，GradientTape 只會紀錄 "trainable variable" 所參與的 operation，因此預設只能計算 loss 相對於 "trainable variable" 的 gradient。

In [24]:
# a trainable variable
x0 = tf.Variable(3.0, name='x0')

# not trainable
x1 = tf.Variable(3.0, trainable=False, name='x1')

# not variable: tf.Variable + tf.Tensor = tf.Tensor
x2 = tf.Variable(3.0, name='x2') + 1.0

# not variable
x3 = tf.constant(3.0, name='x3')

In [25]:
with tf.GradientTape() as tape:
    y = (x0**2) + (x1**2) + (x2**2)

In [26]:
grad = tape.gradient(y, [x0, x1, x2, x3])

In [27]:
for g in grad:
    print(g)

tf.Tensor(6.0, shape=(), dtype=float32)
None
None
None


In [28]:
tape.watched_variables()

(<tf.Variable 'x0:0' shape=() dtype=float32, numpy=3.0>,)

如果要改變 GradientTape 的預設行為，可以先透過 "watch_accessed_variables=False" 取消 GradientTape 關注所有的 "trainable variable"，再透過 GradientTape.watch() 關注特定的元素。

In [29]:
# a trainable variable
x0 = tf.Variable(3.0, name='x0')

# not trainable
x1 = tf.Variable(3.0, trainable=False, name='x1')

# not variable: tf.Variable + tf.Tensor = tf.Tensor
x2 = tf.Variable(3.0, name='x2') + 1.0

In [30]:
with tf.GradientTape(watch_accessed_variables=False) as tape:
    tape.watch(x1)
    tape.watch(x2)
    y = (x0**2) + (x1**2) + (x2**2)

In [31]:
grad = tape.gradient(y, [x0, x1, x2])

In [32]:
grad

[None,
 <tf.Tensor: shape=(), dtype=float32, numpy=6.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=8.0>]

### 計算最終結果相對於「中間值」的 Gradient

In [33]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2
    z = y**2

dz/dy = 2 * y

y = x *  x

In [34]:
grad = tape.gradient(z, y)

In [35]:
grad

<tf.Tensor: shape=(), dtype=float32, numpy=18.0>

### GradientTape Resource Released

在預設情況下，當呼叫 GradientTape.gradient 時，GradientTape 的資源就會被釋放掉，因此沒有辦法呼叫兩次 GradientTape.gradient。如果要針對同一組 Computation 計算多次 Gradient，則在建立 GradientTape 時，必須指定 persistent=True。

In [36]:
x = tf.Variable([1, 2, 3], dtype=tf.float64)

with tf.GradientTape(persistent=True) as tape:
    y = x**2
    z = y**2

dy_dx = 2 * x

In [37]:
tape.gradient(y, x).numpy()

array([2., 4., 6.])

dz_dx = dz_dy * dy_dx

dz_dy = 2 * y</br>
y = x**2</br>
=> y = [1, 4, 9]</br>
=> dz_dy = [2, 8, 18]

dy_dx = 2 * x</br>
=> dy_dx = [2, 4, 6]

In [38]:
tape.gradient(z, x).numpy()

array([  4.,  32., 108.])

記得要將該 tape 刪除，也就是去除 GradientTape 的所有 Reference，GradientTape 佔用的資源才會被釋放

In [39]:
del tape

### 在 GraidentTape 中加入 Control Flow

如果在 GradientTape 中使用 if else 語法，GradientTape 只會紀錄有被執行的 Branch

In [40]:
x = tf.constant(1.0)

v0 = tf.Variable(2.0)
v1 = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
    if x > 0.0:
        result = v0
    else:
        result = v1**2

In [41]:
dv0, dv1 = tape.gradient(result, [v0, v1])

In [49]:
print(dv0)

tf.Tensor(1.0, shape=(), dtype=float32)


In [50]:
print(dv1)

None


### Getting a gradient of none

在以下這些情況中，會得到 gradient = None

#### 1. Target 與 Source 未相連

In [47]:
x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as tape:
    z = y * y

In [48]:
print(tape.gradient(z, x))

None


#### 2. Source 為 tf.Tensor 而非 tf.Variable

GradientTape 在預設情況下只會紀錄 Trainable Variable，如果在更新 tf.Variable 的數值時不是使用 tf.assign()，就會使得 tf.Variable 變成 tf.Tensor

In [54]:
x = tf.Variable(2.0)

with tf.GradientTape() as tape:
    result = x + 1

print(tape.gradient(result, x))

tf.Tensor(1.0, shape=(), dtype=float32)


In [55]:
x = x + 1

with tf.GradientTape() as tape:
    result = x + 1

print(tape.gradient(result, x))

None


#### 3. GradientTape 中的計算不是使用 TensorFlow 的函式

In [57]:
x = tf.Variable([
    [1, 2],
    [3, 4]
])

with tf.GradientTape() as tape:
    x2 = x * x
    y = np.mean(x2)
    

In [58]:
tape.gradient(y, x2)

AttributeError: 'numpy.float64' object has no attribute '_id'

#### 4. Source 為 Integer

In [61]:
x = tf.Variable(1)

with tf.GradientTape() as tape:
    y = x * x

print(tape.gradient(y, x))

None
